### Librerías utilizadas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from xverse.transformer import MonotonicBinning
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score,roc_auc_score, roc_curve
from sklearn.metrics import plot_confusion_matrix, f1_score

# Para poder visualizar todas las columnas:
pd.set_option('display.max_columns',100)

# Para ver unicamente 3 decimales:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Datos utilizados

Se importan los datos que resultaron del proceso de preparación de datos:

In [2]:
data = pd.read_csv('C:/Users/gusta/OneDrive/Documentos/Tesis/Datos/Datos_prep.csv') 

Se seleccionan las variables que serán utilizadas como variables dependientes y se identifica la variable objetivo:

In [3]:
var_predictivas = ['CC','VFA','CC_GRAS','CC_MUSC','CC_HUES','CC_AGUA','PAS',
                   'PAM','PESO','PAD','TALLA','GLU','IMC']
target = 'TARGET_TRI'
X = data[var_predictivas] 
y = data[target] 

### Transformación de las variables: Monotonic Binning

Se transforman las variables numéricas a variables categóricas a través de bins que tengan una relación monótona con la variable objetivo. Durante el proceso se utiliza la correlación de Spearman para verificar si existe una relación monótona entre las variables X e y.

In [4]:
Mbin = MonotonicBinning()
Mbin.fit(X, y)

# Se guardan los bines para aplicar en nuevos datasets
output_bins = Mbin.bins     

A continuación, se muestran los puntos de corte para los bins de cada variable:

In [5]:
print(Mbin.bins)

{'CC': array([ 42.5,  76. ,  85. , 126. ]), 'VFA': array([  5.        ,  44.70000076,  70.80000305, 479.6       ]), 'CC_GRAS': array([ 1.5       , 14.19999981, 21.        , 65.30000305]), 'CC_MUSC': array([ 4.8       ,  7.5       ,  9.30000369, 19.09999466]), 'CC_HUES': array([1.47000003, 2.69000006, 3.22000003, 7.05999994]), 'CC_AGUA': array([17.3       , 27.89999962, 34.40000153, 67.80000305]), 'PAS': array([ 64., 100., 110., 160.]), 'PAM': array([ 48.        ,  80.        ,  86.66666667, 127.33333333]), 'PESO': array([ 38.37      ,  55.34666667,  66.33666667, 127.9       ]), 'PAD': array([ 40.,  70.,  76., 120.]), 'TALLA': array([137., 158., 166., 188.]), 'GLU': array([ 60.,  87.,  94., 220.]), 'IMC': array([14.88857   , 21.368941  , 24.94614033, 43.414966  ])}


Se aplican los bins al dataset X que contine las variables predictivas:

In [6]:
Mbin = MonotonicBinning(custom_binning=output_bins) 

out_X = Mbin.transform(X)
out_X.head()

,CC,VFA,CC_GRAS,CC_MUSC,CC_HUES,CC_AGUA,PAS,PAM,PESO,PAD,TALLA,GLU,IMC
0,"(85.0, 126.0]","(70.8, 479.6]","(21.0, 65.3]","(7.5, 9.3]","(2.69, 3.22]","(27.9, 34.4]","(110.0, 160.0]","(86.667, 127.333]","(66.337, 127.9]","(76.0, 120.0]","(136.999, 158.0]","(87.0, 94.0]","(24.946, 43.415]"
1,"(85.0, 126.0]","(70.8, 479.6]","(21.0, 65.3]","(9.3, 19.1]","(3.22, 7.06]","(34.4, 67.8]","(110.0, 160.0]","(86.667, 127.333]","(66.337, 127.9]","(76.0, 120.0]","(166.0, 188.0]","(87.0, 94.0]","(24.946, 43.415]"
2,"(42.499, 76.0]","(44.7, 70.8]","(1.499, 14.2]","(7.5, 9.3]","(2.69, 3.22]","(27.9, 34.4]","(100.0, 110.0]","(86.667, 127.333]","(38.369, 55.347]","(76.0, 120.0]","(136.999, 158.0]","(59.999, 87.0]","(14.888, 21.369]"
3,"(42.499, 76.0]","(44.7, 70.8]","(1.499, 14.2]","(7.5, 9.3]","(2.69, 3.22]","(27.9, 34.4]","(100.0, 110.0]","(80.0, 86.667]","(38.369, 55.347]","(39.999, 70.0]","(136.999, 158.0]","(59.999, 87.0]","(21.369, 24.946]"
4,"(85.0, 126.0]","(70.8, 479.6]","(21.0, 65.3]","(9.3, 19.1]","(3.22, 7.06]","(34.4, 67.8]","(110.0, 160.0]","(86.667, 127.333]","(66.337, 127.9]","(70.0, 76.0]","(166.0, 188.0]","(94.0, 220.0]","(24.946, 43.415]"


### Transformación de las variables: Weight of Evidence(WOE)

La transformación se basa en el valor logarítmico de las distribuciones. Esto está alineado con la función de salida de regresión logística.  AGREGAR UNA PEQUEÑA DESCRIPCIÓN DEL WOE

In [7]:
## Función WoE
def woe_transform(df,variables,obj):
    for feat in variables:
        aux = {}
        for var in set(df[feat].values):
            PNE = float(len(df[(df[obj]==0) & (df[feat]==var)]))/float(len(df[df[obj]==0]))
            PE = float(len(df[(df[obj]==1) & (df[feat]==var)]))/float(len(df[df[obj]==1]))
            if PNE == 0 or PE == 0:
                woe=0
            else:
                woe=np.log(PNE/PE)
            aux[var]=woe
        df['W_'+feat] = [aux[df[feat][i]] for i in range(0,len(df[feat]))]

Se aplica la transformación WOE en el conjunto de datos:

In [8]:
data_transf = pd.concat([out_X,y], axis=1)

woe_transform(data_transf,var_predictivas, target)
data_transf.head()

,CC,VFA,CC_GRAS,CC_MUSC,CC_HUES,CC_AGUA,PAS,PAM,PESO,PAD,TALLA,GLU,IMC,TARGET_TRI,W_CC,W_VFA,W_CC_GRAS,W_CC_MUSC,W_CC_HUES,W_CC_AGUA,W_PAS,W_PAM,W_PESO,W_PAD,W_TALLA,W_GLU,W_IMC
0,"(85.0, 126.0]","(70.8, 479.6]","(21.0, 65.3]","(7.5, 9.3]","(2.69, 3.22]","(27.9, 34.4]","(110.0, 160.0]","(86.667, 127.333]","(66.337, 127.9]","(76.0, 120.0]","(136.999, 158.0]","(87.0, 94.0]","(24.946, 43.415]",0,-0.72452,-0.68832,-0.64234,0.05700,0.01205,0.04037,-0.61746,-0.43936,-0.64882,-0.42771,0.15786,0.05457,-0.74323
1,"(85.0, 126.0]","(70.8, 479.6]","(21.0, 65.3]","(9.3, 19.1]","(3.22, 7.06]","(34.4, 67.8]","(110.0, 160.0]","(86.667, 127.333]","(66.337, 127.9]","(76.0, 120.0]","(166.0, 188.0]","(87.0, 94.0]","(24.946, 43.415]",1,-0.72452,-0.68832,-0.64234,-0.38664,-0.33779,-0.36859,-0.61746,-0.43936,-0.64882,-0.42771,-0.25147,0.05457,-0.74323
2,"(42.499, 76.0]","(44.7, 70.8]","(1.499, 14.2]","(7.5, 9.3]","(2.69, 3.22]","(27.9, 34.4]","(100.0, 110.0]","(86.667, 127.333]","(38.369, 55.347]","(76.0, 120.0]","(136.999, 158.0]","(59.999, 87.0]","(14.888, 21.369]",0,0.89694,0.17103,0.84876,0.05700,0.01205,0.04037,0.28820,-0.43936,0.94224,-0.42771,0.15786,0.29379,1.24807
3,"(42.499, 76.0]","(44.7, 70.8]","(1.499, 14.2]","(7.5, 9.3]","(2.69, 3.22]","(27.9, 34.4]","(100.0, 110.0]","(80.0, 86.667]","(38.369, 55.347]","(39.999, 70.0]","(136.999, 158.0]","(59.999, 87.0]","(21.369, 24.946]",0,0.89694,0.17103,0.84876,0.05700,0.01205,0.04037,0.28820,0.11121,0.94224,0.25866,0.15786,0.29379,0.14237
4,"(85.0, 126.0]","(70.8, 479.6]","(21.0, 65.3]","(9.3, 19.1]","(3.22, 7.06]","(34.4, 67.8]","(110.0, 160.0]","(86.667, 127.333]","(66.337, 127.9]","(70.0, 76.0]","(166.0, 188.0]","(94.0, 220.0]","(24.946, 43.415]",0,-0.72452,-0.68832,-0.64234,-0.38664,-0.33779,-0.36859,-0.61746,-0.43936,-0.64882,0.26790,-0.25147,-0.33018,-0.74323


### Selección de variables: Information Value (IV)

AGREGAR UNA PEQUEÑA DESCRIPCIÓN DEL IV

In [9]:
## Función IV
def iv_report(df,variables,obj):
    iv_list = []
    for feat in variables:
        lista=[]
        for var in set(df[feat].values):
            PNE = float(len(df[(df[obj]==0) & (df[feat]==var)]))/float(len(df[df[obj]==0]))
            PE = float(len(df[(df[obj]==1) & (df[feat]==var)]))/float(len(df[df[obj]==1]))
            if PNE == 0 or PE == 0:
                woe=0
            else:
                woe=np.log(PNE/PE)
            iv=(PNE-PE)*woe
            lista.append(iv)
        iv_list.append(sum(lista))
    return pd.DataFrame(iv_list, index=variables, columns=['IV'])

Se aplica la función IV para identificar el poder predictivo de las variables consideradas.

In [10]:
iv_tri = iv_report(data_transf,var_predictivas, target)
iv_tri.sort_values(by='IV', ascending=False)

,IV
IMC,0.57367
VFA,0.42975
CC,0.42287
PESO,0.39019
CC_GRAS,0.35653
PAS,0.20775
PAM,0.12366
CC_MUSC,0.11438
PAD,0.11020
CC_AGUA,0.10618


Se puede observar que todas las variables MENCIONAR MÁS COSAS. Se seleccionan las variables que serán utilizadas como variables dependientes y se identifica la variable objetivo:

In [11]:
variables_iv = ['W_IMC','W_VFA', 'W_CC', 'W_PESO', 'W_CC_GRAS', 'W_PAM', 'W_CC_MUSC', 'W_CC_AGUA']
#X = data_transf[[x for x in data_transf.columns if x.startswith('W_')]]
X = data_transf[variables_iv]
y = data_transf[target]

### Separación de datos

Para evaluar mejor el rendimiento del modelo, es una buena estrategia dividir el conjunto de datos dos: un conjunto de entrenamiento (70%) y uno de prueba (30%).

In [12]:
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({0: 1479, 1: 278})


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=19)
print('Train dataset shape %s' % Counter(y_train))
print('Test dataset shape %s' % Counter(y_test))

Train dataset shape Counter({0: 1037, 1: 192})
Test dataset shape Counter({0: 442, 1: 86})


## Modelos supervisados para Clasificación

En esta sección se llevará a cabo la construcción de los modelos supervisados de Machine Learning que calculen la probabilidad de que el paciente tenga su nivel de Triglicéridos por arriba de los 150 mg/dL para posteriormente hacer la clasificación en Evento y No-Evento.

### Regresión logística

In [14]:
# Parámetros para el modelo:
param_dict_lr = dict( multi_class=['ovr', 'multinomial'], 
                     solver=['newton-cg','lbfgs','sag'], 
                     max_iter=range(100,150))

# Crear el objeto regresión logística para clasificación:
lr_clf = LogisticRegression()

# Autotuning del modelo:
grid_lr = GridSearchCV(cv=5, estimator=lr_clf, n_jobs=-1, 
                     param_grid=param_dict_lr ,scoring='roc_auc')
grid_lr.fit(X_train,y_train)
lr_clf = grid_lr.best_estimator_

#rsgrid_lr = RandomizedSearchCV(lr_clf, param_dict_lr, cv=20,
#                                scoring='roc_auc', n_iter=40, n_jobs=-1)
#rsgrid_lr.fit(X_train,y_train)
#lr_clf = rsgrid_lr.best_estimator_

# Entrenar la regresión logística:
lr_clf = lr_clf.fit(X_train,y_train)

In [15]:
# Los mejores parámetros:
grid_lr.best_estimator_
#rsgrid_lr.best_estimator_

LogisticRegression(multi_class='multinomial', solver='newton-cg')

In [16]:
# ROC
print (roc_auc_score(y_score=lr_clf.predict_proba(X_train)[:,1],y_true=y_train))
print (roc_auc_score(y_score=lr_clf.predict_proba(X_test)[:,1],y_true=y_test))

0.7338175024108003
0.664829527517626


In [17]:
print (accuracy_score(y_pred=lr_clf.predict(X_train),y_true=y_train))
print (accuracy_score(y_pred=lr_clf.predict(X_test),y_true=y_test))

0.8437754271765663
0.8371212121212122


### Árbol de decisión

In [18]:
# Parámetros para el modelo
param_dict_tr= dict(criterion=['gini','entropy'], splitter=['best','random'],
                 max_depth=range(2,30), max_features=range(2,8),
                 max_leaf_nodes=range(5,35), min_samples_leaf=[0.05])

# Crear el objeto árbol de decisión para clasificación
tr_clf = DecisionTreeClassifier()

# Autotuning de los modelos
#grid_tr = GridSearchCV(cv=5, estimator=tr_clf, n_jobs=-1, 
#                     param_grid=param_dict_tr ,scoring='roc_auc')
#grid_tr.fit(X_train,y_train)
#tr_clf = grid_tr.best_estimator_

rsgrid_tr = RandomizedSearchCV(tr_clf, param_dict_tr, cv=20,
                                scoring='roc_auc', n_iter=40, n_jobs=-1)
rsgrid_tr.fit(X_train,y_train)
tr_clf = rsgrid_tr.best_estimator_

# Entrenar el árbol de decisión
tr_clf = tr_clf.fit(X_train,y_train)

In [19]:
# Los mejores parámetros
#grid_tr.best_estimator_
rsgrid_tr.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=28, max_features=2,
                       max_leaf_nodes=19, min_samples_leaf=0.05)

### Random Forest

In [20]:
# Parámetros para el modelo
param_dict_rf= dict(n_estimators=range(50,150), criterion=['entropy'],
                 max_depth=range(20,30), max_features=range(2,8),
                 max_leaf_nodes=range(5,20), min_samples_leaf=[0.05])

# Crear el objeto Random Forest para clasificación
rf_clf = RandomForestClassifier()

#Autotuning de los modelos
#grid_rf = GridSearchCV(cv=5, estimator=rf_clf, n_jobs=-1, 
#                     param_grid=param_dict_rf ,scoring='roc_auc')
#grid_rf.fit(X_train,y_train)
#rf_clf = grid_rf.best_estimator_

rsgrid_rf = RandomizedSearchCV(rf_clf, param_dict_rf, cv=20,
                                scoring='roc_auc', n_iter=40, n_jobs=-1)
rsgrid_rf.fit(X_train,y_train)
rf_clf = rsgrid_rf.best_estimator_

# Entrenar el Random Forest
rf_clf = rf_clf.fit(X_train,y_train)

In [21]:
# Los mejores parámetros
#grid_rf.best_estimator_
rsgrid_rf.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=29, max_features=3,
                       max_leaf_nodes=17, min_samples_leaf=0.05,
                       n_estimators=69)

### Gradient Boosting

In [22]:
# Parámetros para el modelo
param_dict_gb= dict(criterion=['friedman_mse', 'mse', 'mae'], 
                    loss=['deviance','exponential'], 
                    learning_rate=[0.05,0.10,0.15,0.20,0.25,0.30,0.5,0.8,1],
                    max_depth=range(2,10), max_features=range(2,8),
                    max_leaf_nodes=range(5,35), min_samples_leaf=[0.05])

# Crear el objeto Gradient Boosting para clasificación
gb_clf = GradientBoostingClassifier()

#Autotuning de los modelos
#grid_gb = GridSearchCV(cv=5, estimator=gb_clf, n_jobs=-1, 
#                     param_grid=param_dict_gb ,scoring='roc_auc')
#grid_gb.fit(X_train,y_train)
#gb_clf = grid_gb.best_estimator_

rsgrid_gb = RandomizedSearchCV(gb_clf, param_dict_gb, cv=20,
                                scoring='roc_auc', n_iter=40, n_jobs=-1)
rsgrid_gb.fit(X_train,y_train)
gb_clf = rsgrid_gb.best_estimator_

# Entrenar el Gradient Boosting
gb_clf = gb_clf.fit(X_train,y_train)

In [23]:
# Los mejores parámetros
#grid_gb.best_estimator_
rsgrid_gb.best_estimator_

GradientBoostingClassifier(criterion='mae', learning_rate=0.05,
                           loss='exponential', max_depth=5, max_features=4,
                           max_leaf_nodes=19, min_samples_leaf=0.05)

### XGBoost

In [24]:
# Parámetros para el modelo
param_dict_xgb= {'eta': [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
                 'max_depth':[3,4,5,6,8,10,12,15],
                 'min_child_weight': [1,3,5,7],
                 'gamma': [0.0,0.1,0.2,0.3,0.4],
                 'colsample_bytree': [0.3,0.4,0.5,0.7],
                 'n_estimators': range(50,150)}

# Crear el objeto Gradient Boosting para clasificación
xgb_clf = XGBClassifier()

#Autotuning de los modelos
#grid_xgb = GridSearchCV(cv=5, estimator=xgb_clf, n_jobs=-1, 
#                     param_grid=param_dict_xgb ,scoring='roc_auc')
#grid_xgb.fit(X_train,y_train)
#xgb_clf = grid_xgb.best_estimator_

rsgrid_xgb = RandomizedSearchCV(xgb_clf, param_dict_xgb, cv=20,
                                scoring='roc_auc', n_iter=40, n_jobs=-1)
rsgrid_xgb.fit(X_train,y_train)
xgb_clf = rsgrid_xgb.best_estimator_

# Entrenar el Gradient Boosting
xgb_clf = xgb_clf.fit(X_train,y_train)

In [25]:
# Los mejores parámetros
#grid_xgb.best_estimator_
rsgrid_xgb.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, eta=0.05, gamma=0.1,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.0500000007, max_delta_step=0, max_depth=4,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=88, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Estadísticos de precisión

A continuación, se muestra la curva ROC de cada uno de los modelos junto con estadístico AUC que indica el área debajo de la curva ROC.

In [26]:
modelos = [{'label':'Decision Tree','modelo':tr_clf,},
           {'label':'Random Forest','modelo':rf_clf,},
           {'label':'Gradient Boosting','modelo':gb_clf,},
           {'label':'XGBoost','modelo':xgb_clf,},
           {'label':'Regresión Logística','modelo':lr_clf,}]

for m in modelos:
    modelo = m['modelo']
    auc = roc_auc_score(y_true=y_test, y_score=modelo.predict_proba(X_test)[:,1])
    print('%s ROC (auc= %0.2f)' % (m['label'], auc))

Decision Tree ROC (auc= 0.68)
Random Forest ROC (auc= 0.69)
Gradient Boosting ROC (auc= 0.68)
XGBoost ROC (auc= 0.69)
Regresión Logística ROC (auc= 0.66)


In [ ]:
modelos = [{'label':'Regresión Logística','modelo':lr_clf,}]

plt.figure(figsize=(6,5))
for m in modelos:
    modelo = m['modelo']
    fpr, tpr, thresholds = roc_curve(y_test, modelo.predict_proba(X_test)[:,1])
    auc = roc_auc_score(y_true=y_test, y_score=modelo.predict_proba(X_test)[:,1])
    plt.plot(fpr,tpr,label='%s ROC (auc= %0.2f)' % (m['label'], auc))
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05]) 
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (using validation data)')
plt.legend(loc="lower right")
plt.show()

También se puede observar la matriz de confusión:

In [ ]:
for m in modelos:
    modelo = m['modelo']
    disp = plot_confusion_matrix(modelo, X_test, y_test,
                                 cmap=plt.cm.Blues,
                                 normalize=None)
    disp.ax_.set_title('Confusion Matrix: %s' % (m['label']))

plt.show()